<a href="https://colab.research.google.com/github/a-fdezv/Machine-Learning-for-Time-Series-with-Python/blob/main/BigQuery_Job_pull_EPA_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
from google.colab import files

project = 'eco-watch-369004' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=eco-watch-369004:US:bquxjob_54035ec_1849ca481e8)
back to BigQuery to edit the query within the BigQuery user interface.

In [5]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_54035ec_1849ca481e8') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT
  date_local,
  county_code,
  site_num,
  latitude,
  longitude,
  parameter_name,
  arithmetic_mean,
  aqi
FROM
  `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary`
WHERE
  state_name = "Washington"
  AND EXTRACT(YEAR FROM date_local) > 2015
UNION ALL
SELECT
  date_local,
  county_code,
  site_num,
  latitude,
  longitude,
  parameter_name,
  arithmetic_mean,
  aqi
FROM
  `bigquery-public-data.epa_historical_air_quality.no2_daily_summary`
WHERE
  state_name = "Washington"
  AND EXTRACT(YEAR FROM date_local) > 2015
UNION ALL
SELECT
  date_local,
  county_code,
  site_num,
  latitude,
  longitude,
  parameter_name,
  arithmetic_mean,
  aqi
FROM
  `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
WHERE
  state_name = "Washington"
  AND EXTRACT(YEAR FROM date_local) > 2015
UNION ALL
SELECT
  date_local,
  county_code,
  site_num,
  latitude,
  longitude,
  parameter_name,
  arithmetic_mean,
  aqi
FROM
  `bigquery-public-data.epa_historical_ai

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_54035ec_1849ca481e8') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

In [4]:
results_duplicated = results[results.duplicated(['county_code','site_num','date_local','latitude','longitude','parameter_name'], keep=False)]
drop_indexes = results_duplicated[results_duplicated.aqi.isnull()]
#print(drop_indexes.index)
vals = results.drop(list(drop_indexes.index))
vals_duplicated_index = vals[vals.duplicated(['county_code','site_num','date_local','latitude','longitude','parameter_name'], keep='first')].index
data = vals.drop(list(vals_duplicated_index))
#dups = data_nodup.duplicated(['county_code','site_num','date_local','latitude','longitude','parameter_name'], keep='first')
data

,date_local,county_code,site_num,latitude,longitude,parameter_name,arithmetic_mean,aqi
0,2016-01-01,005,0002,46.21835,-119.204153,Wind Speed - Resultant,3.875000,<NA>
1,2016-01-01,005,0002,46.21835,-119.204153,Wind Direction - Resultant,217.000000,<NA>
2,2016-01-01,007,0011,47.43061,-120.341950,Wind Speed - Resultant,1.075000,<NA>
3,2016-01-01,007,0011,47.43061,-120.341950,Wind Direction - Resultant,184.625000,<NA>
4,2016-01-01,009,0013,48.29786,-124.624910,Wind Speed - Resultant,12.675000,<NA>
...,...,...,...,...,...,...,...,...
246558,2022-02-24,013,9991,46.20260,-117.953900,Ozone,0.033941,37
246559,2022-02-25,013,9991,46.20260,-117.953900,Ozone,0.034941,41
246560,2022-02-26,013,9991,46.20260,-117.953900,Ozone,0.026294,34
246561,2022-02-27,013,9991,46.20260,-117.953900,Ozone,0.024294,31


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [5]:
data.describe()

,latitude,longitude,arithmetic_mean,aqi
count,195038.000000,195038.000000,195038.000000,79333.000000
mean,47.447662,-121.279680,136.493942,25.691528
std,0.805811,1.891789,1492.456699,22.068051
min,45.616667,-124.624910,-3.000000,0.000000
25%,47.141100,-122.462560,3.337500,12.000000
50%,47.568236,-122.214400,12.800000,22.000000
75%,48.246900,-120.095556,66.666667,34.000000
max,48.950740,-117.257654,35159.283333,908.000000


In [6]:
import numpy as np
import pandas as pd
data['AQSID']=data['county_code'].astype(str) + data['site_num'].astype(str)
data.drop(['county_code','site_num'], axis=1, inplace=True)

In [7]:
data.parameter_name.unique()

array(['Wind Speed - Resultant', 'Wind Direction - Resultant',
       'PM10 Total 0-10um STP', 'Outdoor Temperature',
       'Relative Humidity ', 'Carbon monoxide',
       'PM2.5 - Local Conditions', 'Ozone', 'Barometric pressure',
       'Nitrogen dioxide (NO2)'], dtype=object)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195038 entries, 0 to 246562
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date_local       195038 non-null  dbdate 
 1   latitude         195038 non-null  float64
 2   longitude        195038 non-null  float64
 3   parameter_name   195038 non-null  object 
 4   arithmetic_mean  195038 non-null  float64
 5   aqi              79333 non-null   Int64  
 6   AQSID            195038 non-null  object 
dtypes: Int64(1), dbdate(1), float64(3), object(2)
memory usage: 12.1+ MB


In [9]:
aqsid_pm25=data[data['parameter_name']=='PM2.5 - Local Conditions']['AQSID']
aqsid_pm25=aqsid_pm25.unique()

In [10]:
df=data.loc[data.apply(lambda x: x.AQSID in aqsid_pm25, axis =1)]

In [11]:
df

,date_local,latitude,longitude,parameter_name,arithmetic_mean,aqi,AQSID
2,2016-01-01,47.430610,-120.341950,Wind Speed - Resultant,1.075000,<NA>,0070011
3,2016-01-01,47.430610,-120.341950,Wind Direction - Resultant,184.625000,<NA>,0070011
12,2016-01-01,47.597222,-122.319722,Wind Speed - Resultant,1.575000,<NA>,0330030
13,2016-01-01,47.597222,-122.319722,Wind Direction - Resultant,107.000000,<NA>,0330030
14,2016-01-01,47.568236,-122.308628,Wind Speed - Resultant,1.816667,<NA>,0330080
...,...,...,...,...,...,...,...
246526,2022-01-31,47.568236,-122.308628,Carbon monoxide,0.129167,2,0330080
246530,2022-01-31,47.597222,-122.319722,Carbon monoxide,0.379167,6,0330030
246532,2022-01-31,47.568236,-122.308628,Barometric pressure,1009.875000,<NA>,0330080
246534,2022-02-01,47.568236,-122.308628,Carbon monoxide,0.200000,2,0330080


In [71]:
df_index=df.set_index(['date_local','latitude','longitude','AQSID', 'parameter_name'])
df_unstack = df_index.unstack('parameter_name')

In [72]:
df_unstack

arithmetic_mean                  \
parameter_name                           Barometric pressure Carbon monoxide   
date_local latitude  longitude   AQSID                                         
2016-01-01 45.643360 -122.587370 0110024                 NaN             NaN   
           46.319320 -119.999677 0770005                 NaN             NaN   
           46.380240 -120.332660 0770015                 NaN             NaN   
           46.598056 -120.499167 0770009                 NaN             NaN   
           47.186400 -122.451700 0530029                 NaN             NaN   
...                                                      ...             ...   
2022-01-31 48.399990 -119.518960 0470013                 NaN             NaN   
           48.520590 -122.614280 0570011                 NaN             NaN   
           48.544448 -117.903425 0650005                 NaN             NaN   
2022-02-01 47.568236 -122.308628 0330080                 NaN             0.2   
           47.597222 -122.319722 0330030                 NaN             0.5   

                                                                 \
parameter_name                           Nitrogen dioxide (NO2)   
date_local latitude  longitude   AQSID                            
2016-01-01 45.643360 -122.587370 0110024                    NaN   
           46.319320 -119.999677 0770005                    NaN   
           46.380240 -120.332660 0770015                    NaN   
           46.598056 -120.499167 0770009                    NaN   
           47.186400 -122.451700 0530029                    NaN   
...                                                         ...   
2022-01-31 48.399990 -119.518960 0470013                    NaN   
           48.520590 -122.614280 0570011                    NaN   
           48.544448 -117.903425 0650005                    NaN   
2022-02-01 47.568236 -122.308628 0330080                    NaN   
           47.597222 -122.319722 0330030                    NaN   

                                                                    \
parameter_name                           Outdoor Temperature Ozone   
date_local latitude  longitude   AQSID                               
2016-01-01 45.643360 -122.587370 0110024                 NaN   NaN   
           46.319320 -119.999677 0770005                 NaN   NaN   
           46.380240 -120.332660 0770015           14.041667   NaN   
           46.598056 -120.499167 0770009                 NaN   NaN   
           47.186400 -122.451700 0530029                 NaN   NaN   
...                                                      ...   ...   
2022-01-31 48.399990 -119.518960 0470013           23.458333   NaN   
           48.520590 -122.614280 0570011                 NaN   NaN   
           48.544448 -117.903425 0650005           29.416667   NaN   
2022-02-01 47.568236 -122.308628 0330080                 NaN   NaN   
           47.597222 -122.319722 0330030                 NaN   NaN   

                                                                \
parameter_name                           PM10 Total 0-10um STP   
date_local latitude  longitude   AQSID                           
2016-01-01 45.643360 -122.587370 0110024                   NaN   
           46.319320 -119.999677 0770005                   NaN   
           46.380240 -120.332660 0770015                   NaN   
           46.598056 -120.499167 0770009                  14.0   
           47.186400 -122.451700 0530029                   NaN   
...                                                        ...   
2022-01-31 48.399990 -119.518960 0470013                   NaN   
           48.520590 -122.614280 0570011                   NaN   
           48.544448 -117.903425 0650005                  17.0   
2022-02-01 47.568236 -122.308628 0330080                   NaN   
           47.597222 -122.319722 0330030                   NaN   

                                                                   \
parameter_

In [73]:
df_unstack.columns  = df_unstack.columns.map('_'.join).str.strip('_')

In [74]:
df_unstack

arithmetic_mean_Barometric pressure  \
date_local latitude  longitude   AQSID                                          
2016-01-01 45.643360 -122.587370 0110024                                  NaN   
           46.319320 -119.999677 0770005                                  NaN   
           46.380240 -120.332660 0770015                                  NaN   
           46.598056 -120.499167 0770009                                  NaN   
           47.186400 -122.451700 0530029                                  NaN   
...                                                                       ...   
2022-01-31 48.399990 -119.518960 0470013                                  NaN   
           48.520590 -122.614280 0570011                                  NaN   
           48.544448 -117.903425 0650005                                  NaN   
2022-02-01 47.568236 -122.308628 0330080                                  NaN   
           47.597222 -122.319722 0330030                                  NaN   

                                          arithmetic_mean_Carbon monoxide  \
date_local latitude  longitude   AQSID                                      
2016-01-01 45.643360 -122.587370 0110024                              NaN   
           46.319320 -119.999677 0770005                              NaN   
           46.380240 -120.332660 0770015                              NaN   
           46.598056 -120.499167 0770009                              NaN   
           47.186400 -122.451700 0530029                              NaN   
...                                                                   ...   
2022-01-31 48.399990 -119.518960 0470013                              NaN   
           48.520590 -122.614280 0570011                              NaN   
           48.544448 -117.903425 0650005                              NaN   
2022-02-01 47.568236 -122.308628 0330080                              0.2   
           47.597222 -122.319722 0330030                              0.5   

                                          arithmetic_mean_Nitrogen dioxide (NO2)  \
date_local latitude  longitude   AQSID                                             
2016-01-01 45.643360 -122.587370 0110024                                     NaN   
           46.319320 -119.999677 0770005                                     NaN   
           46.380240 -120.332660 0770015                                     NaN   
           46.598056 -120.499167 0770009                                     NaN   
           47.186400 -122.451700 0530029                                     NaN   
...                                                                          ...   
2022-01-31 48.399990 -119.518960 0470013                                     NaN   
           48.520590 -122.614280 0570011                                     NaN   
           48.544448 -117.903425 0650005                                     NaN   
2022-02-01 47.568236 -122.308628 0330080                                     NaN   
           47.597222 -122.319722 0330030                                     NaN   

                                          arithmetic_mean_Outdoor Temperature  \
date_local latitude  longitude   AQSID                                          
2016-01-01 45.643360 -122.587370 0110024                                  NaN   
           46.319320 -119.999677 0770005                                  NaN   
           46.380240 -120.332660 0770015                            14.041667   
           46.598056 -120.499167 0770009                                  NaN   
           47.186400 -122.451700 0530029                                  NaN   
...                                                                       ...   
2022-01-31 48.399990 -119.518960 0470013                            23.458333   
           48.520590 -122.614280 0570011                                  NaN   
           48.544448 -117.903425 0650005                            29.416667   
20

Dropping unnecessary columns and renaming them

In [76]:
df_unstack.drop(['aqi_Barometric pressure','aqi_Outdoor Temperature','aqi_Relative Humidity ','aqi_Wind Direction - Resultant','aqi_Wind Speed - Resultant'], axis=1,inplace=True)
df_unstack.columns
col={'arithmetic_mean_Barometric pressure': 'Psfc', 'arithmetic_mean_Carbon monoxide': 'CO','arithmetic_mean_Nitrogen dioxide (NO2)': 'NO2','arithmetic_mean_PM10 Total 0-10um STP': 'PM10','arithmetic_mean_PM2.5 - Local Conditions': 'PM25','arithmetic_mean_Outdoor Temperature': 'Tmp','arithmetic_mean_Ozone': 'O3','arithmetic_mean_Relative Humidity ': 'RH','arithmetic_mean_Wind Direction - Resultant': 'wd','arithmetic_mean_Wind Speed - Resultant': 'wspd','aqi_Carbon monoxide':'AQI_CO','aqi_Nitrogen dioxide (NO2)':'AQI_NO2','aqi_Ozone':'AQI_O3','aqi_PM10 Total 0-10um STP':'AQI_PM10','aqi_PM2.5 - Local Conditions':'AQI_PM25'}
df_unstack.rename(columns=col,inplace=True)

In [88]:
df_nona=df_unstack.dropna(thresh=3) # drop any row where if it does not have at least 3 values that are **not** NaN (this is at least one more feature besides PM25 nad AQIPM25)

In [89]:
df_nona

Psfc        CO        NO2  \
date_local latitude  longitude   AQSID                                       
2016-01-01 46.380240 -120.332660 0770015          NaN       NaN        NaN   
           46.598056 -120.499167 0770009          NaN       NaN        NaN   
           47.430610 -120.341950 0070011          NaN       NaN        NaN   
           47.568236 -122.308628 0330080  1015.583333  0.355556  23.395833   
           47.597222 -122.319722 0330030          NaN  0.984211  33.529167   
...                                               ...       ...        ...   
2022-01-31 47.568236 -122.308628 0330080  1009.875000  0.129167   8.086957   
           47.597222 -122.319722 0330030          NaN  0.379167  19.773913   
           47.663962 -117.257654 0630017          NaN       NaN        NaN   
           48.399990 -119.518960 0470013          NaN       NaN        NaN   
           48.544448 -117.903425 0650005          NaN       NaN        NaN   

                                                Tmp        O3  PM10  PM25  \
date_local latitude  longitude   AQSID                                      
2016-01-01 46.380240 -120.332660 0770015  14.041667       NaN   NaN  24.3   
           46.598056 -120.499167 0770009        NaN       NaN  14.0  15.8   
           47.430610 -120.341950 0070011  15.875000       NaN   NaN   8.9   
           47.568236 -122.308628 0330080  34.666667  0.011471   NaN  11.3   
           47.597222 -122.319722 0330030  35.750000       NaN   NaN  16.4   
...                                             ...       ...   ...   ...   
2022-01-31 47.568236 -122.308628 0330080  41.041667  0.025417   NaN   3.2   
           47.597222 -122.319722 0330030  41.958333       NaN   NaN   5.3   
           47.663962 -117.257654 0630017        NaN       NaN  10.0   7.8   
           48.399990 -119.518960 0470013  23.458333       NaN   NaN  11.7   
           48.544448 -117.903425 0650005  29.416667       NaN  17.0  10.4   

                                                 RH          wd      wspd  \
date_local latitude  longitude   AQSID                                      
2016-01-01 46.380240 -120.332660 0770015        NaN  214.208333  1.241667   
           46.598056 -120.499167 0770009        NaN         NaN       NaN   
           47.430610 -120.341950 0070011        NaN  184.625000  1.075000   
           47.568236 -122.308628 0330080  63.083333  213.583333  1.816667   
           47.597222 -122.319722 0330030        NaN  107.000000  1.575000   
...                                             ...         ...       ...   
2022-01-31 47.568236 -122.308628 0330080  87.000000  187.333333  2.554167   
           47.597222 -122.319722 0330030        NaN  189.208333  2.575000   
           47.663962 -117.257654 0630017        NaN         NaN       NaN   
           48.399990 -119.518960 0470013        NaN  171.916667  3.283333   
           48.544448 -117.903425 0650005        NaN  168.250000  1.404167   

                                          AQI_CO  AQI_NO2  AQI_O3  AQI_PM10  \
date_local latitude  longitude   AQSID                                        
2016-01-01 46.380240 -120.332660 0770015    <NA>     <NA>    <NA>      <NA>   
           46.598056 -120.499167 0770009    <NA>     <NA>    <NA>        13   
           47.430610 -120.341950 0070011    <NA>     <NA>    <NA>      <NA>   
           47.568236 -122.308628 0330080       5       37      15      <NA>   
           47.597222 -122.319722 0330030      13       45    <NA>      <NA>   
...                                          ...      ...     ...       ...   
2022-01-31 47.568236 -122.308628 0330080       2       22      30      <NA>   
           47.597222 -122.319722 0330030       6       27    <NA>      <NA>   
           47.663962 -117.257654 0630017    <NA>     <NA>    <NA>         9   
           48.399990 -119.518960 0470013    <NA>     <NA>    <NA>      <NA>   
           48.544448 -117.903425 0650005    <NA>     <NA>    <NA>        16   

  

In [90]:
df_data = df_nona.reset_index() #reset the index

In [91]:
df_data

,date_local,latitude,longitude,AQSID,Psfc,CO,NO2,Tmp,O3,PM10,PM25,RH,wd,wspd,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25
0,2016-01-01,46.380240,-120.332660,0770015,NaN,NaN,NaN,14.041667,NaN,NaN,24.3,NaN,214.208333,1.241667,<NA>,<NA>,<NA>,<NA>,77
1,2016-01-01,46.598056,-120.499167,0770009,NaN,NaN,NaN,NaN,NaN,14.0,15.8,NaN,NaN,NaN,<NA>,<NA>,<NA>,13,59
2,2016-01-01,47.430610,-120.341950,0070011,NaN,NaN,NaN,15.875000,NaN,NaN,8.9,NaN,184.625000,1.075000,<NA>,<NA>,<NA>,<NA>,37
3,2016-01-01,47.568236,-122.308628,0330080,1015.583333,0.355556,23.395833,34.666667,0.011471,NaN,11.3,63.083333,213.583333,1.816667,5,37,15,<NA>,47
4,2016-01-01,47.597222,-122.319722,0330030,NaN,0.984211,33.529167,35.750000,NaN,NaN,16.4,NaN,107.000000,1.575000,13,45,<NA>,<NA>,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20077,2022-01-31,47.568236,-122.308628,0330080,1009.875000,0.129167,8.086957,41.041667,0.025417,NaN,3.2,87.000000,187.333333,2.554167,2,22,30,<NA>,13
20078,2022-01-31,47.597222,-122.319722,0330030,NaN,0.379167,19.773913,41.958333,NaN,NaN,5.3,NaN,189.208333,2.575000,6,27,<NA>,<NA>,22
20079,2022-01-31,47.663962,-117.257654,0630017,NaN,NaN,NaN,NaN,NaN,10.0,7.8,NaN,NaN,NaN,<NA>,<NA>,<NA>,9,33
20080,2022-01-31,48.399990,-119.518960,0470013,NaN,NaN,NaN,23.458333,NaN,NaN,11.7,NaN,171.916667,3.283333,<NA>,<NA>,<NA>,<NA>,49


In [92]:
my_dict={k: g[:] for k,g in df_data.groupby("AQSID")} # create a dictionary where keys are the AQSID sites and the values are the corresponded dataframe with features values from 2016-present.

**Data** **Exploratory**

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
my_dict.keys()

NameError: ignored

In [94]:
my_dict.keys() #Analizing individual site to check for null values. 
for site in my_dict.keys():
  print(site)
  df_aqs = my_dict[str(site)]
  df_aqs.info()
  df_aqs.describe()
  corrM = df_unstack.corr()


0070011
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1887 entries, 2 to 20076
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date_local  1887 non-null   object 
 1   latitude    1887 non-null   float64
 2   longitude   1887 non-null   float64
 3   AQSID       1887 non-null   object 
 4   Psfc        0 non-null      float64
 5   CO          0 non-null      float64
 6   NO2         0 non-null      float64
 7   Tmp         1887 non-null   float64
 8   O3          0 non-null      float64
 9   PM10        0 non-null      float64
 10  PM25        422 non-null    float64
 11  RH          0 non-null      float64
 12  wd          1887 non-null   float64
 13  wspd        1887 non-null   float64
 14  AQI_CO      0 non-null      Int64  
 15  AQI_NO2     0 non-null      Int64  
 16  AQI_O3      0 non-null      Int64  
 17  AQI_PM10    0 non-null      Int64  
 18  AQI_PM25    414 non-null    Int64  
dtypes: Int64(5), float

In [29]:
df_unstack.describe()

,Psfc,CO,NO2,Tmp,O3,PM10,PM25,RH,wd,wspd,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25
count,2174.000000,4006.000000,6412.000000,17058.000000,2937.000000,6882.000000,39186.000000,1927.000000,15885.000000,15905.000000,4003.000000,6412.000000,2937.000000,6882.000000,38579.000000
mean,1004.456574,0.309192,14.702258,52.079481,0.022291,20.065679,7.868245,77.744984,185.215487,3.355486,4.617537,24.466781,26.139598,17.841906,28.596283
std,6.524691,0.152276,6.742637,15.220422,0.008852,22.627932,13.573955,13.009318,46.983886,1.845590,2.604318,9.981702,8.884185,16.052740,24.989668
min,978.333333,0.100000,1.566667,-1.260870,-0.000529,1.000000,-3.000000,22.916667,12.375000,0.312500,1.000000,2.000000,0.000000,1.000000,0.000000
25%,1000.843750,0.200000,9.809091,41.125000,0.016529,10.000000,3.300000,68.687500,157.750000,2.066667,2.000000,18.000000,21.000000,9.000000,14.000000
50%,1004.541667,0.279167,14.193334,51.666667,0.022588,15.000000,5.300000,78.458333,185.750000,2.970833,5.000000,24.000000,27.000000,14.000000,22.000000
75%,1008.489583,0.391667,18.777083,63.583333,0.028647,23.000000,8.500000,88.187500,212.541667,4.200000,6.000000,30.000000,32.000000,21.000000,35.000000
max,1027.750000,1.595455,59.234783,95.875000,0.053235,440.000000,513.400000,100.000000,350.833333,17.387500,25.000000,98.000000,105.000000,320.000000,509.000000


Correlation between features

In [34]:
corrM = df_unstack.corr()
corrM

,Psfc,CO,NO2,Tmp,O3,PM10,PM25,RH,wd,wspd,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25
Psfc,1.000000,0.046873,0.110509,-0.051162,-0.205405,0.060491,0.025235,-0.141013,-0.070313,-0.202098,0.046328,0.034880,-0.208845,0.064019,0.089979
CO,0.046873,1.000000,0.808326,-0.084874,-0.393592,0.564138,0.561069,0.129582,-0.132898,-0.203849,0.942724,0.618200,-0.357195,0.569931,0.659718
NO2,0.110509,0.808326,1.000000,-0.060763,-0.561430,0.526971,0.368263,0.011068,-0.062160,-0.268352,0.780515,0.851743,-0.431835,0.553196,0.552340
Tmp,-0.051162,-0.084874,-0.060763,1.000000,0.165978,0.184349,0.049589,-0.517765,0.166123,0.131905,-0.078542,0.052242,0.253380,0.206638,-0.009453
O3,-0.205405,-0.393592,-0.561430,0.165978,1.000000,-0.152648,-0.130158,-0.384963,0.148179,0.528737,-0.464473,-0.310098,0.929289,-0.163588,-0.224240
PM10,0.060491,0.564138,0.526971,0.184349,-0.152648,1.000000,0.899883,-0.283110,-0.036249,-0.148809,0.524606,0.497767,-0.065521,0.980823,0.815843
PM25,0.025235,0.561069,0.368263,0.049589,-0.130158,0.899883,1.000000,-0.078832,-0.017988,-0.201385,0.484615,0.321287,-0.071963,0.841857,0.886548
RH,-0.141013,0.129582,0.011068,-0.517765,-0.384963,-0.283110,-0.078832,1.000000,-0.314849,-0.156855,0.151255,-0.182540,-0.443261,-0.296156,-0.141865
wd,-0.070313,-0.132898,-0.062160,0.166123,0.148179,-0.036249,-0.017988,-0.314849,1.000000,0.191482,-0.176441,0.002173,0.222015,-0.037544,-0.045738
wspd,-0.202098,-0.203849,-0.268352,0.131905,0.528737,-0.148809,-0.201385,-0.156855,0.191482,1.000000,-0.203846,-0.253906,0.474714,-0.164579,-0.348883


From the above deescription whe can conclude that this problem can be tackle from 2 different point of views:


1.   We can do the forecast for each specific site, where the features used as independant variables to predict the target valu may vary from site to site.
2.   We can do the forecast for te entire WA region  taking all the available data as an all.

From those 2 point of views it seemed more accurate to create one model per AQ site. 

In [62]:
df_unstack.to_csv('/drive/My Drive/ML_HW1/data_EPA_WA_2016_daily.csv')

In [60]:
!cp data_EPA_WA_2016_daily.csv "drive/My Drive/ML_HW1/"